## Steganography

**a) Least Significant Bit Algorithm**

In [45]:
from PIL import Image

def message_to_binary(message):
    binary_message = ''.join(format(ord(char), '08b') for char in message)
    return binary_message

def embed_message(image_path, message, embedded_path):
    img = Image.open(image_path)
    binary_message = message_to_binary(message)
    binary_message += '1111111111111110'  # Add a delimiter to mark the end of the message

    data_index = 0

    width, height = img.size

    # Loop through each pixel in the image
    for y in range(height):
        for x in range(width):
            # Get the pixel value
            pixel = img.getpixel((x, y))
            # Convert pixel values to binary
            binary_pixel = [format(component, '08b') for component in pixel]
            # Embed message into the least significant bit of each color channel
            for i in range(3):  # Loop through RGB channels
                binary_pixel[i] = binary_pixel[i][:-1] + binary_message[data_index]
                data_index += 1
                if data_index >= len(binary_message):
                    break
            
            # Convert back to integer pixel values
            pixel = tuple(int(channel, 2) for channel in binary_pixel)
            img.putpixel((x, y), pixel)

            if data_index >= len(binary_message):
                break
        if data_index >= len(binary_message):
            break

    img.save(embedded_path)
    print("Message embedded successfully.")

image_path = "original_image.png"
embedded_path = "embedded_image.png"
message = "Cats are incredible creatures. They bring joy, comfort, and companionship to our lives. Their playful antics and independent spirit make them beloved members of countless households worldwide. Whether they're curled up in a cozy spot, chasing after toys, or gracefully exploring their surroundings, cats never fail to captivate us with their charm and grace. Their soft purrs and gentle nuzzles have a soothing effect, melting away stress and worries. From their adorable whiskers to their graceful movements, every aspect of a cat is a testament to their unique beauty and personality. In a world full of chaos, cats provide a sense of calm and tranquility that is truly unmatched. They remind us to appreciate the simple joys in life and to cherish the moments spent in their company. So here's to our feline friends, who enrich our lives in more ways than we can count. Meow!"
embed_message(image_path, message, embedded_path)

Message embedded successfully.


Original Image:

<img src="original_image.png" alt="Original Image" width="400"/>

Embedded Image:

<img src="embedded_image.png" alt="Embedded Image" width="400"/>

In [48]:
def extract_message(image_path):
    img = Image.open(image_path)
    extracted_binary = ''
    width, height = img.size

    finish = False

    for y in range(height):
        if(finish):
            break
        for x in range(width):
            if(finish):
                break
            # Get the pixel value
            pixel = img.getpixel((x, y))
            # Convert pixel values to binary
            binary_pixel = [format(component, '08b') for component in pixel]

            for i in range(3):  # Loop through RGB channels
                extracted_binary += binary_pixel[i][-1]
                byte = extracted_binary[i:i+8]
                if byte == '11111111':  # Check for delimiter marking end of message
                    finish = True
                    break

    # Split binary string into 8-bit chunks to convert back to characters
    extracted_message = ''
    for i in range(0, len(extracted_binary), 8):
        byte = extracted_binary[i:i+8]
        if byte == '11111111':  # Check for delimiter marking end of message
            break
        extracted_message += chr(int(byte, 2))

    print("Extracted message:", extracted_message)

# Extract the message from the embedded image
extract_message(embedded_path)

Extracted message: Cats are incredible creatures. They bring joy, comfort, and companionship to our lives. Their playful antics and independent spirit make them beloved members of countless households worldwide. Whether they're curled up in a cozy spot, chasing after toys, or gracefully exploring their surroundings, cats never fail to captivate us with their charm and grace. Their soft purrs and gentle nuzzles have a soothing effect, melting away stress and worries. From their adorable whiskers to their graceful movements, every aspect of a cat is a testament to their unique beauty and personality. In a world full of chaos, cats provide a sense of calm and tranquility that is truly unmatched. They remind us to appreciate the simple joys in life and to cherish the moments spent in their company. So here's to our feline friends, who enrich our lives in more ways than we can count. Meow!


### Questions:
1. Is this method of hiding information in an image resistant to attacks and attempts to destroy the embedded message?
2. Propose attacks on the embedded message.
3. What is the size of the message that we can hide in an image/graphic file?

### Answers:
1. The method of hiding information in an image using least significant bit (LSB) steganography is not inherently resistant to attacks and attempts to destroy the embedded message. Since only the least significant bits of the image pixels are modified, the embedded message can potentially be detected and altered through various means, such as statistical analysis, visual inspection, or targeted attacks aimed at manipulating the LSBs. Moreover, common image processing techniques like compression or resizing can inadvertently alter or remove the embedded message.
   
2. Some potential attacks on the embedded message include:
   - Statistical analysis: Analyzing the frequency distribution of pixel values or LSB changes in the image to detect anomalies indicative of hidden data.
   - Compression: Applying lossy compression algorithms (e.g., JPEG) to the image may introduce artifacts that alter or remove the embedded message.
   - Watermark removal: Attempting to remove watermarks or hidden messages using specialized software or algorithms designed for image manipulation.
   
3. The size of the message that can be hidden in an image or graphic file depends on factors such as the resolution of the image, the bit depth of the color channels, and the amount of available LSBs per pixel. In general, the capacity for hiding data using LSB steganography is limited by the number of pixels in the image and the number of LSBs that can be modified without significantly altering the image's visual quality. As a rough estimate, for a typical high-resolution image with 24-bit color depth (8 bits per channel), the message size that can be hidden may range from a few kilobytes to several megabytes, depending on the image's dimensions and characteristics.


**b) Patchwork Algorithm**

In [67]:
import numpy as np
from PIL import Image

def change_brightness(pixel, delta):
    # Extract RGB components
    r, g, b = pixel
    # Change brightness by delta
    r = max(0, min(255, r + delta))
    g = max(0, min(255, g + delta))
    b = max(0, min(255, b + delta))
    return (r, g, b)

def embed_watermark(image_path, watermarked_path, secret_key, num_iterations, delta):
    # Open the image
    img = Image.open(image_path)
    width, height = img.size

    # Convert the image to a NumPy array
    img_array = np.array(img)

    # Set the random seed based on the secret key
    np.random.seed(secret_key)

    S_prime = (2*delta*num_iterations, 2*delta*num_iterations, 2*delta*num_iterations)
    
    # Perform embedding for each iteration
    for i in range(num_iterations):
        # Generate two random areas A and B
        x_a, y_a = np.random.randint(0, width), np.random.randint(0, height)
        x_b, y_b = np.random.randint(0, width), np.random.randint(0, height)

        S_prime += img_array[y_a, x_a] - img_array[y_b, x_b]

        # Modify the pixel intensities
        img_array[y_a, x_a] = np.clip(img_array[y_a, x_a] + delta, 0, 255)
        img_array[y_b, x_b] = np.clip(img_array[y_b, x_b] - delta, 0, 255)

    # Convert the modified array back to an image
    watermarked_img = Image.fromarray(img_array)
    watermarked_img.save(watermarked_path)

    return S_prime

# Example usage
image_path = "original_image.png"
watermarked_path = "watermark_image.png"
secret_key = 12345
num_iterations = 10
delta = 1

S_prime = embed_watermark(image_path, watermarked_path, secret_key, num_iterations, delta)
print("S_prime:", S_prime)


S_prime: [1651 1398 1043]


In [70]:
import numpy as np
from PIL import Image

def detect_watermark(watermarked_path, secret_key, num_iterations, delta, S_prime):
    # Open the image
    img = Image.open(watermarked_path)
    width, height = img.size

    # Convert the image to a NumPy array
    img_array = np.array(img)

    # Set the random seed based on the secret key
    np.random.seed(secret_key)

    S_n = S_prime
    
    # Undo embedding for each iteration
    for i in range(num_iterations):
        # Generate two random areas A and B
        x_a, y_a = np.random.randint(0, width), np.random.randint(0, height)
        x_b, y_b = np.random.randint(0, width), np.random.randint(0, height)

        # Modify the pixel intensities
        img_array[y_a, x_a] = np.clip(img_array[y_a, x_a] + delta, 0, 255)
        img_array[y_b, x_b] = np.clip(img_array[y_b, x_b] - delta, 0, 255)

        S_n += img_array[y_a, x_a] - img_array[y_b, x_b]
    
    print("S_prime:", S_prime)
    print("S_n:", S_n)

    if(np.array_equal(np.array(S_n), np.array(S_prime))):
        print("Watermark detected.")
    else:
        print("No watermark detected.")

detect_watermark(watermarked_path, secret_key, num_iterations, delta, S_prime)


S_prime: [6664 5652 4232]
S_n: [6664 5652 4232]
Watermark detected.


## Effectiveness and resistance to attacks of the patchwork algorithm:

The security of the watermark relies heavily on the randomness and secrecy of the secret key used for embedding, making it vulnerable to attacks if the key is compromised.